# Deep Learning

## Assignment 3

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [203]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [204]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:

- data as a flat matrix,
- labels as float 1-hot encodings.

In [205]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [206]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

***
### Problem 1

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.
***
#### note: 
> the add `(+)` operator supports broadcasting. If the last axes of the tensors to sum are equal or one of them is 1, tf > aligns them in the last dimension and sums them, and stretches (replicates it) the the smaller tensor so the rest of > the dimensions match. That's why the sum inside tf.nn.relu works, as the result of tf.matmul is in this case a 
> `(batch_size, image_size*image_size)` matrix and the hidden_biases tensor is a `(hidden_layer_size)` column vector.
***

In [202]:
#One hidden layer Neural Network with L2 Regularization


def get_nn_logits(dataset, hidden_weights, hidden_biases, softmax_weights, softmax_biases):
    
    #note that the add (+) operator supports broadcasting. If the last axes of the tensors to sum are
    # equal or one of them is 1, tf aligns them in the last dimension and sums them, and 
    # stretches (replicates it) the the smaller tensor so the rest of the dimensions match.
    #that's why the sum inside tf.nn.relu works, as the result of tf.matmul is in this case
    #a (batch_size, image_size*image_size) matrix and the hidden_biases tensor is a 
    # (hidden_layer_size) column vector
    
    hidden_activations = tf.nn.relu(tf.matmul(dataset, hidden_weights) + hidden_biases)
    logits = tf.matmul(hidden_activations, softmax_weights) + softmax_biases
    return logits

def train_validate_and_test(num_steps = 3001, train_size = 200000):
    batch_size = 128
    hidden_layer_size = 1024
    l2_regularization_penalty = 1e-11
    

    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_hidden_layer = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
      biases_hidden_layer = tf.Variable(tf.zeros([hidden_layer_size]))


      weights_softmax_layer = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
      biases_softmax_layer = tf.Variable(tf.zeros([num_labels]))


      # Training computation.
      logits = get_nn_logits(tf_train_dataset, weights_hidden_layer, biases_hidden_layer,
                            weights_softmax_layer, biases_softmax_layer)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
      + l2_regularization_penalty*(tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(weights_softmax_layer))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)

      logits_valid = get_nn_logits(tf_valid_dataset, weights_hidden_layer, biases_hidden_layer,
                            weights_softmax_layer, biases_softmax_layer)
      valid_prediction = tf.nn.softmax(logits_valid)

      logits_test = get_nn_logits(tf_test_dataset, weights_hidden_layer, biases_hidden_layer,
                            weights_softmax_layer, biases_softmax_layer)
      test_prediction = tf.nn.softmax(logits_test)

    

    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

train_validate_and_test()

Initialized
Minibatch loss at step 0: 383.812714
Minibatch accuracy: 7.8%
Validation accuracy: 28.4%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 64.0%
Test accuracy: 70.7%


***
### Problem 2

Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?
***

In [174]:
train_dataset = train_dataset[:256, :]
train_labels = train_labels[:256, :]
train_validate_and_test()

Initialized
Minibatch loss at step 0: 492.525848
Minibatch accuracy: 7.0%
Validation accuracy: 24.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.4%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.5%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.9%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 69.3%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 68.7%
Test accuracy: 75.5%


***
### Problem 3
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?
***

In [173]:
#One hidden layer Neural Network with L2 Regularization and Dropout


def get_nn_logits(dataset, hidden_weights, hidden_biases, softmax_weights, softmax_biases, keep_prob = 1):
    hidden_activations = tf.nn.relu(tf.matmul(dataset, hidden_weights) + hidden_biases)
    dropout = tf.nn.dropout(hidden_activations, keep_prob)
    logits = tf.matmul(dropout, softmax_weights) + softmax_biases
    return logits

def train_validate_and_test(num_steps = 3001, train_size = 200000):
    batch_size = 128
    hidden_layer_size = 1024
    l2_regularization_penalty = 1e-11
    keep_prob = 0.5

    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
      weights_hidden_layer = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
      biases_hidden_layer = tf.Variable(tf.zeros([hidden_layer_size]))


      weights_softmax_layer = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
      biases_softmax_layer = tf.Variable(tf.zeros([num_labels]))


      # Training computation.
      logits = get_nn_logits(tf_train_dataset, weights_hidden_layer, biases_hidden_layer,
                            weights_softmax_layer, biases_softmax_layer, keep_prob)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
      + l2_regularization_penalty*(tf.nn.l2_loss(weights_hidden_layer) + tf.nn.l2_loss(weights_softmax_layer))

      # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)

      logits_valid = get_nn_logits(tf_valid_dataset, weights_hidden_layer, biases_hidden_layer,
                            weights_softmax_layer, biases_softmax_layer)
      valid_prediction = tf.nn.softmax(logits_valid)

      logits_test = get_nn_logits(tf_test_dataset, weights_hidden_layer, biases_hidden_layer,
                            weights_softmax_layer, biases_softmax_layer)
      test_prediction = tf.nn.softmax(logits_test)

    

    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

train_dataset = train_dataset[:512, :]
train_labels = train_labels[:512, :]
train_validate_and_test()

Initialized
Minibatch loss at step 0: 544.950806
Minibatch accuracy: 16.4%
Validation accuracy: 23.2%
Minibatch loss at step 500: 33.328247
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 19.668045
Minibatch accuracy: 78.1%
Validation accuracy: 79.0%
Minibatch loss at step 1500: 13.023142
Minibatch accuracy: 76.6%
Validation accuracy: 79.3%
Minibatch loss at step 2000: 4.614604
Minibatch accuracy: 79.7%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 5.172779
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 4.339188
Minibatch accuracy: 71.1%
Validation accuracy: 79.4%
Test accuracy: 87.0%


***
### Problem 4

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is 97.1%.

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

`global_step = tf.Variable(0)  # count the number of steps taken.
learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)`
***

In [208]:
#Multiple hidden layers Neural Network with L2 Regularization and Dropout


def get_nn_logits(dataset, weights, biases, keep_prob = 1):
    
    hidden_activations_1 = tf.nn.relu(tf.matmul(dataset, weights['hidden_1']) + biases['hidden_1'])
    dropout_1 = tf.nn.dropout(hidden_activations_1, keep_prob)
    
    hidden_activations_2 = tf.nn.relu(tf.matmul(dropout_1, weights['hidden_2']) + biases['hidden_2'])
    dropout_2 = tf.nn.dropout(hidden_activations_2, keep_prob)
    
    hidden_activations_3 = tf.nn.relu(tf.matmul(dropout_2, weights['hidden_3']) + biases['hidden_3'])
    dropout_3 = tf.nn.dropout(hidden_activations_3, keep_prob)
    
    logits = tf.matmul(dropout_3, weights['softmax']) + biases['softmax']
    return logits

def train_validate_and_test(num_steps = 1001, train_size = 200000):
    batch_size = 2048
    hidden_layer_1_size = 4096
    hidden_layer_2_size = 2048
    hidden_layer_3_size = 1024
    l2_regularization_penalty = 1e-2
    keep_prob = 0.5

    graph = tf.Graph()
    with graph.as_default():

      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
      tf_test_dataset = tf.constant(test_dataset)

      # Variables.
    
      weights = {
          'hidden_1' : tf.get_variable(name='w1', shape=[image_size * image_size, hidden_layer_1_size],
                                       initializer=tf.contrib.layers.xavier_initializer()),
          'hidden_2' : tf.get_variable(name='w2', shape=[hidden_layer_1_size, hidden_layer_2_size],
                                       initializer=tf.contrib.layers.xavier_initializer()),
          'hidden_3' : tf.get_variable(name='w3', shape=[hidden_layer_2_size, hidden_layer_3_size],
                                   initializer=tf.contrib.layers.xavier_initializer()),
          'softmax' : tf.get_variable(name='softmax', shape=[hidden_layer_3_size, num_labels],
                                  initializer=tf.contrib.layers.xavier_initializer())
      }
      
      biases = {
          'hidden_1': tf.Variable(tf.zeros([hidden_layer_1_size])),
          'hidden_2': tf.Variable(tf.zeros([hidden_layer_2_size])),
          'hidden_3': tf.Variable(tf.zeros([hidden_layer_3_size])),
          'softmax': tf.Variable(tf.zeros([num_labels])),
      }

      # Training computation.
      logits = get_nn_logits(tf_train_dataset, weights, biases, keep_prob)
      loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
      + l2_regularization_penalty*(tf.nn.l2_loss(weights['hidden_1']) + tf.nn.l2_loss(weights['hidden_2'])
      + tf.nn.l2_loss(weights['hidden_3'])+ tf.nn.l2_loss(weights['softmax']))

      # Optimizer.
      #global_step = tf.Variable(0, trainable=False)
      #starter_learning_rate = 0.0001
      #learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 10, 0.96, staircase=True)
      optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)

      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits)

      logits_valid = get_nn_logits(tf_valid_dataset, weights, biases)
      valid_prediction = tf.nn.softmax(logits_valid)

      logits_test = get_nn_logits(tf_test_dataset, weights, biases)
      test_prediction = tf.nn.softmax(logits_test)

    

    with tf.Session(graph=graph) as session:
      tf.global_variables_initializer().run()
      print("Initialized")
      for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

train_validate_and_test(num_steps = 2001)

Initialized
Minibatch loss at step 0: 2.409302
Minibatch accuracy: 11.5%
Validation accuracy: 47.1%
Minibatch loss at step 500: 0.356648
Minibatch accuracy: 89.1%
Validation accuracy: 88.2%
Minibatch loss at step 1000: 0.347385
Minibatch accuracy: 89.0%
Validation accuracy: 89.8%
Minibatch loss at step 1500: 0.304388
Minibatch accuracy: 90.8%
Validation accuracy: 90.6%
Minibatch loss at step 2000: 0.256214
Minibatch accuracy: 92.2%
Validation accuracy: 91.0%
Test accuracy: 96.5%
